In [31]:
from keras.src.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

In [32]:
text_pairs = []

with open("ara.txt", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines:
    parts = line.strip().split("\t")
    if len(parts) < 2:
        continue
    eng = parts[0]
    ara = parts[1]
    ara = "[start] " + ara + " [end]"
    text_pairs.append((eng, ara))
for _ in range(5):
    print(random.choice(text_pairs))

('Any student can answer that question.', '[start] أي طالب يمكنه الرد على هذا السؤال. [end]')
('Tom did it in his own way.', '[start] توم فعل ذلك بطريقته الخاصة. [end]')
("He's stupid and dishonest.", '[start] هو غبي وغير أمين. [end]')
("They're washing their hands.", '[start] إنهم يغسلون أيديهم. [end]')
("Fighting won't settle anything.", '[start] لن يحل الشجار شيئاً. [end]')


In [34]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

12523 total pairs
8767 training pairs
1878 validation pairs
1878 test pairs


In [35]:
chars=string.punctuation
chars = chars.replace("[", "")
chars = chars.replace("]", "")
vocab = 15000
seq = 20


def standard(input):
    lowercase = tf_strings.lower(input)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(chars), "")


en_vectorizer=TextVectorization(max_tokens=vocab,output_sequence_length=seq,output_mode="int",)
train_eng=[pair[0] for pair in train_pairs]
en_vectorizer.adapt(train_eng)

ar_vectorizer=TextVectorization(max_tokens=vocab,output_sequence_length=seq+1,output_mode="int",standardize=standard,)
train_ar=[pair[1] for pair in train_pairs]
ar_vectorizer.adapt(train_ar)



In [36]:
def format_dataset(en, ar):
    en = en_vectorizer(en)
    ar = ar_vectorizer(ar)
    return (
        {
            "encoder_inputs": en,
            "decoder_inputs": ar[:, :-1],
        },
        ar[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(64)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)



In [37]:
for batch in train_ds.take(1):
    inputs, targets = batch
    print("Encoder inputs:", inputs["encoder_inputs"])
    print("Decoder inputs:", inputs["decoder_inputs"])
    print("Targets:", targets)


Encoder inputs: tf.Tensor(
[[  53 3703    3 ...    0    0    0]
 [   3  104   51 ...    0    0    0]
 [  13   97    7 ...    0    0    0]
 ...
 [ 544  150 3711 ...    0    0    0]
 [   2   95   61 ...    0    0    0]
 [  18  207    4 ...    0    0    0]], shape=(64, 20), dtype=int64)
Decoder inputs: tf.Tensor(
[[   2 6845    3 ...    0    0    0]
 [   2   83   32 ...    0    0    0]
 [   2  415   20 ...    0    0    0]
 ...
 [   2 1775   81 ...    0    0    0]
 [   2   12 1161 ...    0    0    0]
 [   2  182   16 ...    0    0    0]], shape=(64, 20), dtype=int64)
Targets: tf.Tensor(
[[6845    3    0 ...    0    0    0]
 [  83   32 9457 ...    0    0    0]
 [ 415   20 1964 ...    0    0    0]
 ...
 [1775   81 9479 ...    0    0    0]
 [  12 1161    6 ...    0    0    0]
 [ 182   16  279 ...    0    0    0]], shape=(64, 20), dtype=int64)


In [38]:
import keras.ops as ops

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [39]:
class Encoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.norm1 = layers.LayerNormalization()
        self.norm2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padmask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padmask = None

        att_output = self.att(
            query=inputs, value=inputs, key=inputs, attention_mask=padmask
        )
        dense_input = self.norm1(inputs + att_output)
        dense_output = self.dense(dense_input)
        return self.norm2(dense_input + dense_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config



In [40]:
class Decoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.att1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.att2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.norm_1 = layers.LayerNormalization()
        self.norm_2 = layers.LayerNormalization()
        self.norm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        inputs, encoder_outputs = inputs
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is None:
            inputs_padding_mask, encoder_outputs_padding_mask = None, None
        else:
            inputs_padding_mask, encoder_outputs_padding_mask = mask

        att_out_1 = self.att1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask,
            query_mask=inputs_padding_mask,
        )
        out_1 = self.norm_1(inputs + att_out_1)

        att_out_2 = self.att2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            query_mask=inputs_padding_mask,
            key_mask=encoder_outputs_padding_mask,
        )
        out_2 = self.norm_2(out_1 + att_out_2)

        proj_output = self.dense(out_2)
        return self.norm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [41]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(seq, vocab, embed_dim)(encoder_inputs)
encoder_outputs = Encoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(seq, vocab, embed_dim)(decoder_inputs)
x = Decoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

transformer = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)

In [42]:
transformer.compile("rmsprop",loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),metrics=["accuracy"],
)
transformer.fit(train_ds, epochs=20, validation_data=val_ds)

Epoch 1/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 830s 6s/step - accuracy: 0.0497 - loss: 7.4628 - val_accuracy: 0.0621 - val_loss: 6.4509
Epoch 2/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 817s 6s/step - accuracy: 0.0613 - loss: 6.4279 - val_accuracy: 0.0721 - val_loss: 6.1536
Epoch 3/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 817s 6s/step - accuracy: 0.0769 - loss: 5.8350 - val_accuracy: 0.0787 - val_loss: 5.8830
Epoch 4/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 845s 6s/step - accuracy: 0.0894 - loss: 5.2794 - val_accuracy: 0.0923 - val_loss: 5.5360
Epoch 5/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 820s 6s/step - accuracy: 0.1015 - loss: 4.7921 - val_accuracy: 0.0936 - val_loss: 5.2643
Epoch 6/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 861s 6s/step - accuracy: 0.1116 - loss: 4.3196 - val_accuracy: 0.0961 - val_loss: 5.2295
Epoch 7/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 808s 6s/step - accuracy: 0.1211 - loss: 3.9746 - val_accuracy: 0.1012 - val_loss: 4.9797
Epoch 8/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 800s 6s/step - accuracy: 0.1312 - loss: 3.5825 - 

KeyboardInterrupt: 

In [43]:
ar_vocab = ar_vectorizer.get_vocabulary()
spa_index_lookup = dict(zip(range(len(ar_vocab)), ar_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = en_vectorizer([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ar_vectorizer([decoded_sentence])[:, :-1]
        predictions = transformer(
            {
                "encoder_inputs": tokenized_input_sentence,
                "decoder_inputs": tokenized_target_sentence,
            }
        )

        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)

In [68]:
transformer.save("translator_en_ar.keras")

In [81]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading

app = Flask(__name__)
CORS(app)



@app.route("/translate", methods=["POST"])
def translate():
    data = request.get_json()
    sentence = data.get("sentence", "")
    if not sentence:
        return jsonify({"error": "No sentence provided"}), 400

    translation = decode_sequence(sentence)
    clean_translation = translation.replace("[start]", "").replace("[end]", "").strip()

    return jsonify({"translation": clean_translation})

def run_flask():
    app.run(port=5000)

threading.Thread(target=run_flask).start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [79]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 5s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [80]:
!python3 your_flask_file.py
!npx localtunnel --port 5000

python3: can't open file '/content/your_flask_file.py': [Errno 2] No such file or directory
⠙⠹⠸⠼⠴⠦your url is: https://breezy-waves-play.loca.lt
^C


In [ ]:
!npx localtunnel --port 5000

⠙⠹⠸⠼⠴⠦your url is: https://real-hotels-listen.loca.lt
